# Solution

In [16]:
import pandas as pd
import pickle

class CustomerDataExtractor:
    CATEGORY_MAP = {
        1: 'Electronics',
        2: 'Apparel',
        3: 'Books',
        4: 'Home Goods'
    }

    def __init__(self, customer_data_path: str, vip_list_path: str):
        self.customer_data_path = customer_data_path
        self.vip_list_path = vip_list_path
        self.vip_ids = self._load_vip_ids()
        self.data = self._load_customer_data()

    def _load_vip_ids(self):
        with open(self.vip_list_path, 'r') as file:
            return set(int(line.strip()) for line in file.readlines() if line.strip().isdigit())

    def _load_customer_data(self):
        with open(self.customer_data_path, 'rb') as file:
            return pickle.load(file)

    def flatten_data(self):
        records = []

        for customer in self.data:
            customer_id = customer.get('id')
            customer_name = customer.get('name')
            registration_date = self._safe_to_datetime(customer.get('registration_date'), 'registration_date')  # NEW
            is_vip = customer_id in self.vip_ids

            orders = customer.get('orders', [])
            if isinstance(orders, list):
                for order in orders:
                    order_id = order.get('order_id')
                    order_date = self._safe_to_datetime(order.get('order_date'), 'order_date')  # NEW
                    order_total_value = abs(self._safe_to_float(order.get('order_total_value')))  # NEW: make positive


                    items = order.get('items', [])
                    if isinstance(items, list):
                        for item in items:
                            category_raw = item.get('category')
                            category = self.CATEGORY_MAP.get(category_raw, 'Misc')

                            price = item.get('price')
                            quantity = item.get('quantity')

                            price_f = self._safe_to_float(price)  # CHANGED
                            quantity_i = self._safe_to_int(quantity)  # CHANGED
                            total_item_price = price_f * quantity_i if price_f is not None and quantity_i is not None else None

                            percentage = (total_item_price / order_total_value * 100) if total_item_price and order_total_value else 0.0

                            records.append({
                                'customer_id': self._safe_to_int(customer_id),  # CHANGED
                                'customer_name': customer_name,
                                'registration_date': registration_date,
                                'is_vip': is_vip,
                                'order_id': self._safe_to_int(order_id),  # CHANGED
                                'order_date': order_date,
                                'product_id': self._safe_to_int(item.get('item_id')),  # CHANGED
                                'product_name': item.get('product_name'),
                                'category': category,
                                'unit_price': price_f,
                                'item_quantity': quantity_i,
                                'total_item_price': total_item_price,
                                'total_order_value_percentage': percentage
                            })

        df = pd.DataFrame(records)

        df = df.astype({
            'customer_id': 'int64',
            'customer_name': 'string',
            'registration_date': 'datetime64[ns]',
            'is_vip': 'bool',
            'order_id': 'int64',
            'order_date': 'datetime64[ns]',
            'product_id': 'int64',
            'product_name': 'string',
            'category': 'string',
            'unit_price': 'float',
            'item_quantity': 'int64',
            'total_item_price': 'float',
            'total_order_value_percentage': 'float'
        })

        df.sort_values(by=['customer_id', 'order_id', 'product_id'], inplace=True)
        df.reset_index(drop=True, inplace=True)

        return df

    def run_all(self, output_csv_path=None):
        df = self.flatten_data()
        if output_csv_path:
            df.to_csv(output_csv_path, index=False)
        return df

    def _safe_to_int(self, value): 
        try:
            val = str(value).replace('O', '0').replace('$', '').replace(',', '').strip()
            val = val.replace('–', '-')  
            if val in ['-', '–', '']:
                raise ValueError("Invalid isolated minus")
            if '.' in val:
                val = val.split('.')[0]  
            return int(val)
        except (ValueError, TypeError):
            # print(f"Warning: could not convert to int: {value!r}")
            return 0


    def _safe_to_float(self, value):  # UPDATED: added '-' cleaning
        try:
            val_str = str(value).replace('O', '0').replace('$', '').replace(',', '').strip()
            val_str = val_str.replace('–', '-')  # EN dash to minus
            if val_str in ['-', '–', '']:
                raise ValueError("Invalid isolated minus")
            return float(val_str)
        except (ValueError, TypeError):
            # print(f"Warning: could not convert to float: {value!r}")
            return 0.0


    def _safe_to_datetime(self, value, col_name=''):  # NEW
        try:
            return pd.to_datetime(value, errors='coerce')
        except Exception:
            # print(f"Warning: could not convert to datetime ({col_name}): {value!r}")
            return pd.NaT

# https://chatgpt.com/c/685d3a90-06b8-800a-8699-1c93322282dd
# https://chatgpt.com/c/685d43ae-875c-800a-bc3e-dd4e4fdf695e
if __name__ == "__main__":
    extractor = CustomerDataExtractor("customer_orders.pkl", "vip_customers.txt")
    df_final = extractor.run_all("resulted_data.csv")
    print(df_final.dtypes)
    print(df_final.shape)


customer_id                              int64
customer_name                   string[python]
registration_date               datetime64[ns]
is_vip                                    bool
order_id                                 int64
order_date                      datetime64[ns]
product_id                               int64
product_name                    string[python]
category                        string[python]
unit_price                             float64
item_quantity                            int64
total_item_price                       float64
total_order_value_percentage           float64
dtype: object
(198, 13)


In [17]:
df_final.describe(include="all")

,customer_id,customer_name,registration_date,is_vip,order_id,order_date,product_id,product_name,category,unit_price,item_quantity,total_item_price,total_order_value_percentage
count,198.000000,198,193,198,198.000000,192,198.000000,198,198,198.000000,198.000000,198.000000,198.000000
unique,NaN,38,NaN,2,NaN,NaN,NaN,196,5,NaN,NaN,NaN,NaN
top,NaN,Customer 18,NaN,False,NaN,NaN,NaN,,Misc,NaN,NaN,NaN,NaN
freq,NaN,17,NaN,163,NaN,NaN,NaN,2,88,NaN,NaN,NaN,NaN
mean,25.207071,NaN,2023-01-22 14:28:11.471502592,NaN,42.292929,2024-04-20 02:40:16.510416640,2.141414,NaN,NaN,251.237727,2.782828,695.063384,68.992245
min,1.000000,NaN,2022-01-15 19:06:56,NaN,0.000000,2022-02-11 14:16:49,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000
25%,14.500000,NaN,2022-06-04 23:35:47,NaN,15.250000,2023-10-21 05:21:52,1.000000,NaN,NaN,123.602500,1.000000,225.355000,9.789068
50%,22.000000,NaN,2022-09-22 19:35:51,NaN,37.000000,2024-07-04 01:52:04,2.000000,NaN,NaN,247.740000,3.000000,518.720000,29.217228
75%,38.000000,NaN,2023-12-02 08:35:54,NaN,66.750000,2025-01-03 16:45:51,3.000000,NaN,NaN,378.327500,4.000000,983.670000,52.451252
max,50.000000,NaN,2024-04-29 09:54:31,NaN,99.000000,2025-06-04 22:19:31,5.000000,NaN,NaN,499.780000,5.000000,2498.900000,2211.182748



Fixed values with $, commas, and OCR errors like O instead of 0.
Properly handled negative signs (including different dash types).
Changed negative order totals to positive values to keep percentage calculations meaningful (but you can also choose to set them to zero based on your needs).
Detected and warned about invalid entries like 'FREE', 'INVALID', and non-numeric order IDs (e.g., 'ORD14').
Safely converted invalid or malformed dates to missing values without errors.

# Draft

In [42]:
import pandas as pd
import pickle
with open('customer_orders.pkl' , 'rb') as i:
    c = pickle.load(i)
df = pd.DataFrame(c)
print(df.shape)

(50, 4)


In [43]:
df['orders'][0]

[{'order_id': 84,
  'order_date': '2024-02-27 21:24:16',
  'shipping_address': '840 Main St, City 8',
  'order_total_value': 0,
  'items': []},
 {'order_id': None,
  'order_date': '2025-05-03 08:09:20',
  'shipping_address': '533 Main St, City 9',
  'order_total_value': 2765.8,
  'items': [{'item_id': 1,
    'product_name': 'Item 1 for Order 2',
    'category': 4,
    'price': 52.7,
    'quantity': 2},
   {'item_id': 2,
    'product_name': 'Item 2 for Order 2',
    'category': 5,
    'price': 498.45,
    'quantity': 5},
   {'item_id': 3,
    'product_name': 'Item 3 for Order 2',
    'category': 5,
    'price': 168.15,
    'quantity': 1}]},
 {'order_id': 3,
  'order_date': '2023-09-06 00:42:50',
  'shipping_address': '588 Main St, City 3',
  'order_total_value': 0,
  'items': [{'item_id': 1,
    'product_name': 'Item 1 for Order 3',
    'category': 1,
    'price': '$377.96',
    'quantity': 1}]}]

In [44]:
import pandas as pd

all_items = []
# https://chatgpt.com/c/685d3474-d66c-800a-96cc-cb7fb3788146
for idx, row in df.iterrows():
    customer_id = row['id']
    customer_name = row['name']
    registration_date = row['registration_date']
    orders = row['orders']

    if isinstance(orders, list):
        for order in orders:
            order_id = order.get('order_id')
            order_date = order.get('order_date')
            shipping_address = order.get('shipping_address')
            order_total_value = order.get('order_total_value')
            items = order.get('items', [])

            if isinstance(items, list):
                for item in items:
                    flat_item = {
                        'customer_id': customer_id,
                        'customer_name': customer_name,
                        'registration_date': registration_date,
                        'order_id': order_id,
                        'order_date': order_date,
                        'shipping_address': shipping_address,
                        'order_total_value': order_total_value,
                        'item_id': item.get('item_id'),
                        'product_name': item.get('product_name'),
                        'category': item.get('category'),
                        'price': item.get('price'),
                        'quantity': item.get('quantity'),
                    }
                    all_items.append(flat_item)

items_df = pd.DataFrame(all_items)
print(items_df.shape)
print(items_df.head())

(198, 12)
   customer_id customer_name    registration_date order_id  \
0            1    Customer 1  2022-12-31 04:19:19     None   
1            1    Customer 1  2022-12-31 04:19:19     None   
2            1    Customer 1  2022-12-31 04:19:19     None   
3            1    Customer 1  2022-12-31 04:19:19        3   
4            2    Customer 2  2022-05-27 00:23:28        5   

            order_date     shipping_address  order_total_value  item_id  \
0  2025-05-03 08:09:20  533 Main St, City 9        2765.800000      1.0   
1  2025-05-03 08:09:20  533 Main St, City 9        2765.800000      2.0   
2  2025-05-03 08:09:20  533 Main St, City 9        2765.800000      3.0   
3  2023-09-06 00:42:50  588 Main St, City 3           0.000000      1.0   
4  2024-08-28 19:37:56  848 Main St, City 3         -57.165786      1.0   

         product_name category    price quantity  
0  Item 1 for Order 2        4     52.7        2  
1  Item 2 for Order 2        5   498.45        5  
2  Item 3 for

In [47]:
items_df.describe(include="all")

,customer_id,customer_name,registration_date,order_id,order_date,shipping_address,order_total_value,item_id,product_name,category,price,quantity
count,198.000000,198,193,184.0,197,198,197.000000,183.000000,198,198.0,196,198.0
unique,NaN,38,37,64.0,67,69,NaN,NaN,196,14.0,193,7.0
top,NaN,Customer 18,2023-12-07 02:31:09,37.0,2025-03-11 16:22:57,"989 Main St, City 8",NaN,NaN,,1.0,INVALID,1.0
freq,NaN,17,17,5.0,5,5,NaN,NaN,2,36.0,4,46.0
mean,25.207071,NaN,NaN,NaN,NaN,NaN,2281.656619,2.316940,NaN,NaN,NaN,NaN
std,14.680093,NaN,NaN,NaN,NaN,NaN,1524.009285,1.248334,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN,-95.205725,1.000000,NaN,NaN,NaN,NaN
25%,14.500000,NaN,NaN,NaN,NaN,NaN,1287.360000,1.000000,NaN,NaN,NaN,NaN
50%,22.000000,NaN,NaN,NaN,NaN,NaN,2066.440000,2.000000,NaN,NaN,NaN,NaN
75%,38.000000,NaN,NaN,NaN,NaN,NaN,3146.180000,3.000000,NaN,NaN,NaN,NaN


In [48]:
# %%time
date_cols = []
for column in items_df.select_dtypes(include=['object']).columns:
    temp_conversion = pd.to_datetime(items_df[column], errors='coerce') # If conversion produces only NaTs, it is a date
    if not temp_conversion.isnull().all():  
        date_cols.append(column)
date_cols


/var/folders/zd/2p6zdy652y56pz5qz0hmdskc0000gn/T/ipykernel_38751/3442935656.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  temp_conversion = pd.to_datetime(items_df[column], errors='coerce') # If conversion produces only NaTs, it is a date
/var/folders/zd/2p6zdy652y56pz5qz0hmdskc0000gn/T/ipykernel_38751/3442935656.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  temp_conversion = pd.to_datetime(items_df[column], errors='coerce') # If conversion produces only NaTs, it is a date
/var/folders/zd/2p6zdy652y56pz5qz0hmdskc0000gn/T/ipykernel_38751/3442935656.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as

['registration_date',
 'order_id',
 'order_date',
 'category',
 'price',
 'quantity']

In [49]:
items_df[['registration_date',
 'order_id',
 'order_date',
 'category',
 'price',
 'quantity']].isna().sum()

registration_date     5
order_id             14
order_date            1
category              0
price                 2
quantity              0
dtype: int64

In [50]:
items_df.describe(include="all")

,customer_id,customer_name,registration_date,order_id,order_date,shipping_address,order_total_value,item_id,product_name,category,price,quantity
count,198.000000,198,193,184.0,197,198,197.000000,183.000000,198,198.0,196,198.0
unique,NaN,38,37,64.0,67,69,NaN,NaN,196,14.0,193,7.0
top,NaN,Customer 18,2023-12-07 02:31:09,37.0,2025-03-11 16:22:57,"989 Main St, City 8",NaN,NaN,,1.0,INVALID,1.0
freq,NaN,17,17,5.0,5,5,NaN,NaN,2,36.0,4,46.0
mean,25.207071,NaN,NaN,NaN,NaN,NaN,2281.656619,2.316940,NaN,NaN,NaN,NaN
std,14.680093,NaN,NaN,NaN,NaN,NaN,1524.009285,1.248334,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN,-95.205725,1.000000,NaN,NaN,NaN,NaN
25%,14.500000,NaN,NaN,NaN,NaN,NaN,1287.360000,1.000000,NaN,NaN,NaN,NaN
50%,22.000000,NaN,NaN,NaN,NaN,NaN,2066.440000,2.000000,NaN,NaN,NaN,NaN
75%,38.000000,NaN,NaN,NaN,NaN,NaN,3146.180000,3.000000,NaN,NaN,NaN,NaN


In [7]:
import pandas as pd
import pickle
from collections import Counter

class CustomerDataExtractor:
    CATEGORY_MAP = {
        1: 'Electronics',
        2: 'Apparel',
        3: 'Books',
        4: 'Home Goods'
    }

    def __init__(self, customer_data_path: str, vip_list_path: str):
        self.customer_data_path = customer_data_path
        self.vip_list_path = vip_list_path
        self.vip_ids = self._load_vip_ids()
        self.data = self._load_customer_data()

    def _load_vip_ids(self):
        with open(self.vip_list_path, 'r') as file:
            return set(int(line.strip()) for line in file.readlines() if line.strip().isdigit())

    def _load_customer_data(self):
        with open(self.customer_data_path, 'rb') as file:
            return pickle.load(file)

    def flatten_data(self):
        records = []

        for customer in self.data:
            customer_id = customer.get('id')
            customer_name = customer.get('name')
            registration_date = self._safe_to_datetime(customer.get('registration_date'), 'registration_date')
            is_vip = customer_id in self.vip_ids

            orders = customer.get('orders', [])
            if isinstance(orders, list):
                for order in orders:
                    order_id = order.get('order_id')
                    order_date = self._safe_to_datetime(order.get('order_date'), 'order_date')
                    order_total_value = order.get('order_total_value', 0)

                    items = order.get('items', [])
                    if isinstance(items, list):
                        for item in items:
                            category_raw = item.get('category')
                            category = self.CATEGORY_MAP.get(category_raw, 'Misc')

                            price = item.get('price')
                            quantity = item.get('quantity')

                            price_f = self._to_float(price)
                            quantity_i = self._to_int(quantity)
                            total_item_price = price_f * quantity_i if price_f is not None and quantity_i is not None else None

                            percentage = (total_item_price / order_total_value * 100) if total_item_price and order_total_value else 0.0

                            records.append({
                                'customer_id': customer_id,
                                'customer_name': customer_name,
                                'registration_date': registration_date,
                                'is_vip': is_vip,
                                'order_id': order_id,
                                'order_date': order_date,
                                'product_id': item.get('item_id'),
                                'product_name': item.get('product_name'),
                                'category': category,
                                'unit_price': price,
                                'item_quantity': quantity,
                                'total_item_price': total_item_price,
                                'total_order_value_percentage': percentage
                            })

        # Run inspection on all columns before type conversion
        self.inspect_dirty_values(records)

        # Create DataFrame
        df = pd.DataFrame(records)

        # Convert types safely with explicit print-on-fail
        df['customer_id'] = df['customer_id'].apply(self._safe_to_int)
        df['customer_name'] = df['customer_name'].astype('string')
        df['registration_date'] = df['registration_date'].apply(self._safe_to_datetime_apply)
        df['is_vip'] = df['is_vip'].astype('bool')
        df['order_id'] = df['order_id'].apply(self._safe_to_int)
        df['order_date'] = df['order_date'].apply(self._safe_to_datetime_apply)
        df['product_id'] = df['product_id'].apply(self._safe_to_int)
        df['product_name'] = df['product_name'].astype('string')
        df['category'] = df['category'].astype('string')
        df['unit_price'] = df['unit_price'].apply(self._safe_to_float)
        df['item_quantity'] = df['item_quantity'].apply(self._safe_to_int)
        df['total_item_price'] = df['total_item_price'].apply(self._safe_to_float)
        df['total_order_value_percentage'] = df['total_order_value_percentage'].apply(self._safe_to_float)

        df.sort_values(by=['customer_id', 'order_id', 'product_id'], inplace=True)
        df.reset_index(drop=True, inplace=True)

        return df

    def run_all(self, output_csv_path=None):
        df = self.flatten_data()
        if output_csv_path:
            df.to_csv(output_csv_path, index=False)
        return df

    def _safe_to_int(self, value):
        try:
            return int(str(value).replace('O', '0').replace('$', '').strip())
        except (ValueError, TypeError):
            print(f"Warning: could not convert to int: {value!r}")
            return 0
    def _to_int(self, value):
        try:
            return int(str(value).replace('O', '0'))  # OCR fix
        except (ValueError, TypeError):
            return 0

    def _to_float(self, value):
        try:
            return float(str(value).replace('O', '0'))
        except (ValueError, TypeError):
            return 0.0

    def _safe_to_float(self, value):
        try:
            val_str = str(value).replace('O', '0').replace('$', '').replace(',', '').strip()
            return float(val_str)
        except (ValueError, TypeError):
            print(f"Warning: could not convert to float: {value!r}")
            return 0.0

    def _safe_to_datetime(self, value, col_name=''):
        try:
            return pd.to_datetime(value)
        except Exception:
            print(f"Warning: could not convert to datetime ({col_name}): {value!r}")
            return pd.NaT

    def _safe_to_datetime_apply(self, value):
        # This is for pandas Series.apply where col_name not known
        try:
            return pd.to_datetime(value)
        except Exception:
            print(f"Warning: could not convert to datetime: {value!r}")
            return pd.NaT

    def inspect_dirty_values(self, records):
        if not records:
            print("No records to inspect.")
            return

        print("\n--- Inspecting columns for unusual / inconsistent values ---\n")

        columns = records[0].keys()

        for col in columns:
            values = [rec.get(col) for rec in records if rec.get(col) is not None]

            if not values:
                print(f"Column '{col}' is empty, skipping.\n")
                continue

            types = [type(v) for v in values]
            type_counts = Counter(types)

            majority_type, majority_count = type_counts.most_common(1)[0]

            unusual_vals = [v for v in values if not isinstance(v, majority_type)]

            if majority_type == str:
                has_digits = [any(c.isdigit() for c in str(v)) for v in values]
                majority_digit_flag = Counter(has_digits).most_common(1)[0][0]
                unusual_vals.extend([v for v, hd in zip(values, has_digits) if hd != majority_digit_flag])
                unusual_vals = list(set(unusual_vals))

            if majority_type in [int, float]:
                unusual_vals.extend([v for v in values if isinstance(v, str)])
                unusual_vals = list(set(unusual_vals))

            if unusual_vals:
                print(f"Column '{col}' has {len(unusual_vals)} unusual values (type or format different from majority):")
                print(" Sample unusual values:")
                for val in unusual_vals[:10]:
                    print(f"  - {val!r}")
                print()
            else:
                print(f"Column '{col}' looks consistent (all mostly {majority_type.__name__}).\n")

# https://chatgpt.com/c/685d3fdf-4bb8-800a-a98a-42b804969eed
if __name__ == "__main__":
    extractor = CustomerDataExtractor("customer_orders.pkl", "vip_customers.txt")
    df_final = extractor.run_all("a.csv")
    # print(df_final.dtypes)



--- Inspecting columns for unusual / inconsistent values ---

Column 'customer_id' looks consistent (all mostly int).

Column 'customer_name' looks consistent (all mostly str).

Column 'registration_date' looks consistent (all mostly Timestamp).

Column 'is_vip' looks consistent (all mostly bool).

Column 'order_id' has 5 unusual values (type or format different from majority):
 Sample unusual values:
  - 'ORD68'
  - 'ORD14'
  - 'ORD57'
  - 'ORD90'
  - 'ORD84'

Column 'order_date' has 5 unusual values (type or format different from majority):
 Sample unusual values:
  - NaT
  - NaT
  - NaT
  - NaT
  - NaT

Column 'product_id' looks consistent (all mostly int).

Column 'product_name' has 2 unusual values (type or format different from majority):
 Sample unusual values:
  - ''
  - '   '

Column 'category' looks consistent (all mostly str).

Column 'unit_price' has 7 unusual values (type or format different from majority):
 Sample unusual values:
  - '$327.13'
  - '$413.40'
  - '$377.96'